# Lab 2: Deep Learning with PyTorch

**AI/ML for Data Scientists - Day 2**

In this lab, you'll practice:
- Building neural networks with PyTorch
- Training and evaluating models
- Using transfer learning with Hugging Face

---

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## Part 1: Create a Simple Neural Network

Let's build a neural network for binary classification.

In [ ]:
# Generate synthetic data
X, y = make_classification(
    n_samples=2000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    random_state=42
)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_t = torch.FloatTensor(X_train)
y_train_t = torch.FloatTensor(y_train)
X_test_t = torch.FloatTensor(X_test)
y_test_t = torch.FloatTensor(y_test)

# Create data loaders
train_dataset = TensorDataset(X_train_t, y_train_t)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

### Exercise 1.1: Define the Neural Network

Create a neural network with:
- Input layer (20 features)
- Hidden layer 1: 64 units, ReLU activation
- Hidden layer 2: 32 units, ReLU activation
- Output layer: 1 unit, Sigmoid activation

In [ ]:
# YOUR CODE HERE: Define the neural network
class BinaryClassifier(nn.Module):
    def __init__(self, input_size):
        super(BinaryClassifier, self).__init__()
        self.layer1 = nn.Linear(input_size, 64)
        self.layer2 = nn.Linear(64, 32)
        self.output = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(0.2)
    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.dropout(x)
        x = self.relu(self.layer2(x))
        x = self.dropout(x)
        x = self.sigmoid(self.output(x))
        return x

# Initialize model
model = BinaryClassifier(input_size=20).to(device)
print(model)

### Exercise 1.2: Train the Model

In [ ]:
# YOUR CODE HERE: Set up loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 50
train_losses = []

for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    
    for batch_X, batch_y in train_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        # Forward pass
        outputs = model(batch_X).squeeze()
        loss = criterion(outputs, batch_y)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")

### Exercise 1.3: Plot Training Loss

In [ ]:
# YOUR CODE HERE: Plot the training loss
plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Over Time')
plt.legend()
plt.grid(True)
plt.show()

### Exercise 1.4: Evaluate the Model

In [ ]:
# YOUR CODE HERE: Evaluate on test set
model.eval()
with torch.no_grad():
    X_test_t = X_test_t.to(device)
    y_pred_proba = model(X_test_t).squeeze().cpu().numpy()
    y_pred = (y_pred_proba >= 0.5).astype(int)

# Calculate metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Test Set Results:")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")

## Part 2: Transfer Learning with Hugging Face

Let's use a pre-trained model for text classification.

In [ ]:
# Note: Run this cell only if transformers is installed
# !pip install transformers

try:
    from transformers import pipeline
    
    # Load sentiment analysis pipeline
    classifier = pipeline("sentiment-analysis")
    
    # Test sentences
    texts = [
        "I love this product! It works amazingly well.",
        "This is terrible. Complete waste of money.",
        "It's okay, nothing special but does the job.",
        "Absolutely fantastic experience, highly recommend!"
    ]
    
    print("Sentiment Analysis Results:")
    print("-" * 50)
    for text in texts:
        result = classifier(text)[0]
        print(f"Text: {text[:50]}...")
        print(f"Sentiment: {result['label']} (confidence: {result['score']:.4f})")
        print()
        
except ImportError:
    print("Transformers library not installed.")
    print("Run: pip install transformers")

## Part 3: Building a CNN for Image Classification (Conceptual)

Here's the structure of a CNN for image classification.

In [ ]:
# CNN architecture example
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        
        # Pooling
        self.pool = nn.MaxPool2d(2, 2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, num_classes)
        
        # Activation and dropout
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        # Conv block 1
        x = self.pool(self.relu(self.conv1(x)))
        # Conv block 2
        x = self.pool(self.relu(self.conv2(x)))
        # Conv block 3
        x = self.pool(self.relu(self.conv3(x)))
        
        # Flatten
        x = x.view(x.size(0), -1)
        
        # Fully connected
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        
        return x

# Model summary
cnn_model = SimpleCNN(num_classes=10)
print(cnn_model)
print(f"\nTotal parameters: {sum(p.numel() for p in cnn_model.parameters()):,}")

## Summary

In this lab, you learned how to:

1. **Build neural networks** with PyTorch
2. **Train models** with forward/backward passes
3. **Evaluate models** on test data
4. **Use transfer learning** with Hugging Face
5. **Understand CNN architecture** for image classification

---

*AI/ML for Data Scientists | AI Elevate*